# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
new_data = data.groupby(['CustomerID', 'ProductName'])['Quantity'].sum()
new_data.head()

CustomerID  ProductName                  
33          Apricots - Dried                 1
            Assorted Desserts                1
            Bandage - Flexible Neon          1
            Bar Mix - Pina Colada, 355 Ml    1
            Beans - Kidney, Canned           1
Name: Quantity, dtype: int64

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(data, values= 'Quantity', index = ['ProductName'],
                       columns = ['CustomerID'],aggfunc=np.sum, fill_value=0 )
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
Apricots - Halves,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,25,50,25,0,25,25
Apricots Fresh,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25
Arizona - Green Tea,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,25,0,0,0,0
Artichokes - Jerusalem,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), 
                         index=table.columns, columns=table.columns)
distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [7]:
similarities = distances[464].sort_values(ascending=False)[1:6]
similarities.index


Int64Index([3317, 3535, 2503, 1008, 639], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [8]:
sim5=data.loc[data['CustomerID'].isin(similarities.index)]
sim5.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
201,1008,Leticia,Krueger,4468566,270,Cheese Cloth No 100,1
242,3535,Lana,Peters,2399506,270,Cheese Cloth No 100,1
1038,2503,Kyle,Reeves,182355,417,"Lamb - Pieces, Diced",1
1042,3535,Lana,Peters,4503728,417,"Lamb - Pieces, Diced",1
1362,3535,Lana,Peters,2575012,159,Eggplant - Asian,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [16]:


products = sim5.groupby(['ProductName'])[['Quantity']].sum().sort_values('Quantity',ascending=False)
products.head()

,Quantity
ProductName,
Butter - Unsalted,4
"Sauce - Gravy, Au Jus, Mix",3
"Pepper - Black, Whole",3
"Peas - Pigeon, Dry",3
Cheese - Cambozola,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [17]:
c464=table[[464]]
products=c464.merge(products, on='ProductName')
products.head()

,464,Quantity
ProductName,,
Anchovy Paste - 56 G Tube,0,2
Appetizer - Mushroom Tart,0,2
Appetizer - Sausage Rolls,0,1
Apricots - Dried,0,1
Assorted Desserts,0,1


In [20]:
prod_sort=products[products[464]==0].sort_values(by='Quantity', ascending=False).head()
prod_sort

,464,Quantity
ProductName,,
Butter - Unsalted,0,4
"Peas - Pigeon, Dry",0,3
Yoghurt Tubes,0,3
"Sauce - Gravy, Au Jus, Mix",0,3
Sausage - Breakfast,0,3


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [29]:
dict = {}
unique_costumers = set(data['CustomerID'])

for c in unique_costumers:
    similarities = (distances[c].sort_values(ascending=False)[1:]).head(10)
    sim5=data.loc[data['CustomerID'].isin(similarities.index)]
    products = sim5.groupby(['ProductName'])[['Quantity']].sum().sort_values('Quantity',ascending=False)
    dict.update(products)
    
recs = dict


##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [30]:
recs = pd.DataFrame(recs)
recs.head(15)

,Quantity
ProductName,
Appetizer - Mushroom Tart,33
V8 - Berry Blend,26
Wonton Wrappers,25
Fenngreek Seed,23
Cheese - Taleggio D.o.p.,23
Juice - V8 Splash,23
Chicken - Wieners,20
Pecan Raisin - Tarts,18
Pasta - Angel Hair,18


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [32]:
distances2 = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'cityblock'))), 
                         index=table.columns, columns=table.columns)

dict2 = {}
unique_costumers = set(data['CustomerID'])

for c in unique_costumers:
    similarities2 = (distances2[c].sort_values(ascending=False)[1:]).head(10)
    sim5_2=data.loc[data['CustomerID'].isin(similarities2.index)]
    products2 = sim5_2.groupby(['ProductName'])[['Quantity']].sum().sort_values('Quantity',ascending=False)
    dict2.update(products)
    
recs = pd.DataFrame(dict2)


In [33]:
recs.head()

,Quantity
ProductName,
Appetizer - Mushroom Tart,33
V8 - Berry Blend,26
Wonton Wrappers,25
Fenngreek Seed,23
Cheese - Taleggio D.o.p.,23
